# Modifed example of the hyp-sdk Notebook


# Using the HyP3 SDK for Python

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ASFHyP3/hyp3-sdk/main?filepath=docs%2Fsdk_example.ipynb)

HyP3's Python SDK `hyp3_sdk` provides a convenience wrapper around the HyP3 API and HyP3 jobs.


The HyP3 SDK can be installed using [Anaconda/Miniconda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/download.html#anaconda-or-miniconda)
 (recommended) via [`conda`](https://anaconda.org/conda-forge/hyp3_sdk):

```
conda install -c conda-forge hyp3_sdk
```

Or using [`pip`](https://pypi.org/project/hyp3-sdk/):

```
python -m pip install hyp3_sdk
```

Full documentation of the SDK can be found in the [HyP3 documentation](https://hyp3-docs.asf.alaska.edu/using/sdk/).

### Other recommended packages

We also recommend installing the `asf_search` Python package for performing searches of the ASF catalog. The ASF Search
Python package can be installed using [Anaconda/Miniconda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/download.html#anaconda-or-miniconda)
(recommended) via [`conda`](https://anaconda.org/conda-forge/asf_search):

```
conda install -c conda-forge asf_search
```

Or using [`pip`](https://pypi.org/project/asf-search):

```
python -m pip install asf_search
```

Full documentation of `asf_search` can be found in the [ASF search documentation](https://docs.asf.alaska.edu/asf_search/basics).

In [5]:
!pip install  hyp3_sdk

# !conda install -c conda-forge hyp3_sdk -y

In [7]:
# initial setup
import asf_search as asf
import hyp3_sdk as sdk

## Authenticating to the API

The SDK will attempt to pull your [NASA Earthdata Login](https://urs.earthdata.nasa.gov/) credentials out of `~/.netrc`
by default, or you can pass your credentials in directly

In [8]:
# .netrc
hyp3 = sdk.HyP3()

AuthenticationError: Was not able to authenticate with .netrc file and no credentials provided
This could be due to invalid credentials in .netrc or a connection error.

In [9]:
# or enter your credentials
hyp3 = sdk.HyP3(prompt=True)

NASA Earthdata Login username:  karisuvtol
NASA Earthdata Login password:  ········


## Submitting jobs

The SDK provides a submit method for [all supported job types](https://hyp3-docs.asf.alaska.edu/products/).

### Submitting Sentinel-1 RTC jobs

Sentinel-1 Radiometric Terrain Correction (RTC) jobs are submitted using [ESA granule IDs](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-1-sar/naming-conventions).
The example granules below can be viewed  in [ASF Search here](https://search.asf.alaska.edu/#/?zoom=7.08772014623877&center=-141.733866,58.498008&resultsLoaded=true&granule=S1A_IW_SLC__1SDV_20210214T154835_20210214T154901_036588_044C54_8494-SLC&searchType=List%20Search&searchList=S1A_IW_SLC__1SDV_20210214T154835_20210214T154901_036588_044C54_8494-SLC,S1B_IW_SLC__1SDV_20210210T153131_20210210T153159_025546_030B48_B568-SLC,S1A_IW_SLC__1SDV_20210210T025526_20210210T025553_036522_0449E2_7769-SLC,S1A_IW_SLC__1SDV_20210210T025501_20210210T025528_036522_0449E2_3917-SLC,S1B_IW_SLC__1SDV_20210209T030255_20210209T030323_025524_030A8D_7E88-SLC,S1B_IW_SLC__1SDV_20210209T030227_20210209T030257_025524_030A8D_5BAF-SLC,S1A_IW_SLC__1SDV_20210202T154835_20210202T154902_036413_044634_01A1-SLC).

In [10]:
granules = [
    'S1A_IW_SLC__1SDV_20210214T154835_20210214T154901_036588_044C54_8494',
    'S1B_IW_SLC__1SDV_20210210T153131_20210210T153159_025546_030B48_B568',
    'S1A_IW_SLC__1SDV_20210210T025526_20210210T025553_036522_0449E2_7769',
    'S1A_IW_SLC__1SDV_20210210T025501_20210210T025528_036522_0449E2_3917',
    'S1B_IW_SLC__1SDV_20210209T030255_20210209T030323_025524_030A8D_7E88',
    'S1B_IW_SLC__1SDV_20210209T030227_20210209T030257_025524_030A8D_5BAF',
    'S1A_IW_SLC__1SDV_20210202T154835_20210202T154902_036413_044634_01A1',
]


rtc_jobs = sdk.Batch()
for g in granules:
    rtc_jobs += hyp3.submit_rtc_job(g, name='rtc-example')
print(rtc_jobs)

7 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 7 pending.


Here we've given each job the name `rtc-example`, which we can use later to search for these jobs.

`HyP3.submit_rtc_job` also accepts
[keyword arguments](https://hyp3-docs.asf.alaska.edu/using/sdk_api/#hyp3_sdk.hyp3.HyP3.submit_rtc_job)
to customize the RTC products to your application.

### Submitting Sentinel-1 InSAR jobs

The SDK can also submit Sentinel-1 Interferometric Synthetic Aperture Radar (InSAR) jobs which processes
reference and secondary granule pairs.

For a particular reference granule, we may want to use the nearest and next-nearest temporal neighbor granules as secondary
scenes. To programmatically find our secondary granules for a reference granule, We'll define a `get_nearest_neighbors`
function that uses the [baseline stack](https://docs.asf.alaska.edu/asf_search/ASFProduct/#stack) method from `asf_search`:

In [11]:
from typing import Optional

def get_nearest_neighbors(granule: str, max_neighbors: Optional[int] = None) -> asf.ASFSearchResults:
    granule = asf.granule_search(granule)[-1]
    stack = reversed([item for item in granule.stack() if item.properties['temporalBaseline'] < 0])
    return asf.ASFSearchResults(stack)[:max_neighbors]

Now, using the example granule list for our RTC jobs as the reference scenes, we can find their nearest and next-nearest neighbor granules, and submit them
as pairs for InSAR processing.

In [ ]:
from tqdm.auto import tqdm  # For a nice progress bar: https://github.com/tqdm/tqdm#ipython-jupyter-integration

insar_jobs = sdk.Batch()
for reference in tqdm(granules):
    neighbors = get_nearest_neighbors(reference, max_neighbors=2)
    for secondary in neighbors:
        insar_jobs += hyp3.submit_insar_job(reference, secondary.properties['sceneName'], name='insar-example')
print(insar_jobs)

Like RTC jobs, `HyP3.submit_insar_job` accepts
[keyword arguments](https://hyp3-docs.asf.alaska.edu/using/sdk_api/#hyp3_sdk.hyp3.HyP3.submit_insar_job)
to customize the InSAR products to your application.

### Submitting autoRIFT jobs

AutoRIFT supports processing Sentinel-1, Sentinel-2, or Landsat-8 Collection 2 pairs.
* Sentinel-1 jobs are submitted using [ESA granule IDs](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-1-sar/naming-conventions)
* Sentinel-2 jobs are submitted using [ESA granule IDs](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/naming-convention)
* Landsat-8 Collection 2 jobs are submitted using [USGS scene IDs](https://www.usgs.gov/faqs/what-naming-convention-landsat-collection-2-level-1-and-level-2-scenes?qt-news_science_products=0#qt-news_science_products)

In [27]:
autorift_pairs = [
    # Sentinel-1 ESA granule IDs
    ('S1A_IW_SLC__1SSH_20170221T204710_20170221T204737_015387_0193F6_AB07',
     'S1B_IW_SLC__1SSH_20170227T204628_20170227T204655_004491_007D11_6654'),
    # Sentinel-2 ESA granule IDs
    ('S2B_MSIL1C_20200612T150759_N0209_R025_T22WEB_20200612T184700',
     'S2A_MSIL1C_20200627T150921_N0209_R025_T22WEB_20200627T170912'),
    # Landsat 8
    ('LC08_L1TP_009011_20200703_20200913_02_T1',
     'LC08_L1TP_009011_20200820_20200905_02_T1'),
]


autorift_pairs = [
    # Sentinel-1 ESA granule IDs
    ('S1A_IW_SLC__1SSH_20210308T082926_20210308T082953_036904_045743_2075',
'S1A_IW_SLC__1SSH_20210320T082926_20210320T082953_037079_045D60_4C31'),

]


autorift_jobs = sdk.Batch()
for reference, secondary in autorift_pairs:
    autorift_jobs += hyp3.submit_autorift_job(reference, secondary, name='autorift-example')
print(autorift_jobs)

1 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 1 pending.


AutoRIFT does not currently accept any keyword arguments for product customization.

## Monitoring jobs

One jobs are submitted, you can either watch the jobs until they finish

In [28]:
rtc_jobs = hyp3.watch(rtc_jobs)

  0%|          | 0/7 [timeout in 10800 s]

which will require you to keep the cell/terminal running, or you can come back later and search for jobs

In [29]:
rtc_jobs = hyp3.find_jobs(name='rtc-example')
rtc_jobs = hyp3.watch(rtc_jobs)

  0%|          | 0/7 [timeout in 10800 s]

### Downloading files

Batches are collections of jobs. They provide a snapshot of the job status when the job was created or last
refreshed. To get updated information on a batch

In [30]:
print(autorift_jobs)
autorift_jobs = hyp3.refresh(autorift_jobs)
print(autorift_jobs)

1 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 1 pending.
1 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 1 pending.


In [31]:
autorift_jobs = hyp3.watch(autorift_jobs)

  0%|          | 0/1 [timeout in 10800 s]

`hyp3.watch()` will return a refreshed batch once every job in the batch has completed.

Batches can be added together

In [17]:
print(f'Number of Jobs:\n  RTC:{len(rtc_jobs)}\n  InSAR:{len(insar_jobs)}\n  autoRIFT:{len(autorift_jobs)}')
all_jobs = rtc_jobs + insar_jobs + autorift_jobs
print(f'Total number of Jobs: {len(all_jobs)}')

NameError: name 'insar_jobs' is not defined

You can check the status of a batch (at last refresh) by printing the batch

In [32]:
print(autorift_jobs)

1 HyP3 Jobs: 1 succeeded, 0 failed, 0 running, 0 pending.


and filter jobs by status

In [33]:
succeeded_jobs = autorift_jobs.filter_jobs(succeeded=True, running=False, failed=False)
print(f'Number of succeeded jobs: {len(autorift_jobs)}')
failed_jobs = autorift_jobs.filter_jobs(succeeded=False, running=False, failed=True)
print(f'Number of failed jobs: {len(failed_jobs)}')

Number of succeeded jobs: 1
Number of failed jobs: 0


You can download the files for all successful jobs

In [34]:
file_list = succeeded_jobs.download_files()

  0%|          | 0/1 [00:00<?, ?it/s]

S1A_IW_SLC__1SSH_20210308T082926_20210308T082953_036904_045743_2075_X_S1A_IW_SLC__1SSH_20210320T082926_2021032…

*Note: only succeeded jobs will have files to download.*


In [35]:
file_list

[PosixPath('S1A_IW_SLC__1SSH_20210308T082926_20210308T082953_036904_045743_2075_X_S1A_IW_SLC__1SSH_20210320T082926_20210320T082953_037079_045D60_4C31_G0120V02_P005_IL_ASF_OD.nc')]